In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

# Set MLflow Tracking URI (adjust if different)
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient()

# Define experiment name
experiment_name = "Housing-Regression"
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    raise ValueError(f"Experiment '{experiment_name}' not found.")

# Fetch all runs
runs = client.search_runs(experiment.experiment_id, order_by=["metrics.r2_score DESC"])

if not runs:
    raise RuntimeError("No runs found in the experiment.")

# Best run (highest R2)
best_run = runs[0]
best_run_id = best_run.info.run_id
r2_score = best_run.data.metrics.get("r2_score")
print(f"✅ Best Run ID: {best_run_id}, R² Score: {r2_score}")

# Model artifact path
model_uri = f"runs:/{best_run_id}/model"

# Register model
model_name = "Housing-BestModel"
try:
    client.create_registered_model(model_name)
    print(f"📦 Created registered model: {model_name}")
except mlflow.exceptions.RestException:
    print(f"ℹ️ Model {model_name} already exists.")

# Create a new model version
model_version = client.create_model_version(
    name=model_name,
    source=model_uri,
    run_id=best_run_id
)

# Transition to production
client.transition_model_version_stage(
    name=model_name,
    version=model_version.version,
    stage="Production",
    archive_existing_versions=True,
)

print(f"🎯 Promoted model version {model_version.version} to 'Production'.")

# Confirm production model load
loaded_model = mlflow.sklearn.load_model(f"models:/{model_name}/Production")
print("✅ Production model is ready for inference.")
